# LSTM 기반 신경 신호 분류 및 예측

이 노트북은 적응형 신경 전기자극 시스템에서 LSTM(Long Short-Term Memory) 네트워크를 사용하여 신경 신호를 분류하고 예측하는 방법을 설명합니다. LSTM은 시계열 데이터 처리에 탁월한 성능을 보이는 순환 신경망(RNN)의 한 종류로, 신경 신호와 같은 시간에 따른 패턴을 학습하는 데 이상적입니다.

## 1. 필요한 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import sys

# 시각화 설정
plt.style.use('seaborn-whitegrid')
sns.set_theme(style="whitegrid")

# 경로 설정
sys.path.append('..')
from utils.data_utils import load_neural_data, preprocess_neural_signals

# 랜덤 시드 설정
np.random.seed(42)
tf.random.set_seed(42)

## 2. 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
data_path = '../data/neural_recordings/'
signals_data = load_neural_data(data_path)

# 데이터 확인
print(f"데이터 형태: {signals_data['signals'].shape}")
print(f"레이블 수: {len(signals_data['labels'])}")
print(f"레이블 클래스: {np.unique(signals_data['labels'])}")

In [ ]:
# 데이터 전처리
X, y, feature_names = preprocess_neural_signals(signals_data['signals'], signals_data['labels'])

# 데이터 분할 (훈련, 검증, 테스트)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# 데이터 정규화
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# LSTM을 위한 3D 형태로 변환 [samples, time steps, features]
# 여기서는 각 시계열을 10개의 타임스텝으로 분할합니다
def reshape_for_lstm(X, time_steps):
    samples, features = X.shape
    # 타임스텝으로 나누어지는 샘플 수 계산
    valid_samples = samples // time_steps * time_steps
    # 유효한 샘플만 선택
    X_valid = X[:valid_samples]
    # LSTM 형태로 재구성
    X_reshaped = X_valid.reshape(valid_samples // time_steps, time_steps, features)
    return X_reshaped

time_steps = 10
X_train_lstm = reshape_for_lstm(X_train_scaled, time_steps)
X_val_lstm = reshape_for_lstm(X_val_scaled, time_steps)
X_test_lstm = reshape_for_lstm(X_test_scaled, time_steps)

# 레이블도 동일하게 변환
y_train_lstm = y_train[:len(X_train_lstm) * time_steps:time_steps]
y_val_lstm = y_val[:len(X_val_lstm) * time_steps:time_steps]
y_test_lstm = y_test[:len(X_test_lstm) * time_steps:time_steps]

print(f"LSTM 훈련 데이터 형태: {X_train_lstm.shape}")
print(f"LSTM 검증 데이터 형태: {X_val_lstm.shape}")
print(f"LSTM 테스트 데이터 형태: {X_test_lstm.shape}")

## 3. LSTM 모델 구축

In [ ]:
def build_lstm_model(input_shape, num_classes):
    model = Sequential([
        # 첫 번째 LSTM 층
        Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape),
        Dropout(0.3),
        
        # 두 번째 LSTM 층
        Bidirectional(LSTM(32)),
        Dropout(0.3),
        
        # 출력 층
        Dense(16, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    
    return model

# 입력 형태 및 클래스 수 정의
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])
num_classes = len(np.unique(y))

# 모델 생성
lstm_model = build_lstm_model(input_shape, num_classes)
lstm_model.summary()

## 4. 모델 훈련

In [ ]:
# 콜백 정의
checkpoint_path = "../models/lstm_neural_signal_model.h5"
checkpoint = ModelCheckpoint(
    checkpoint_path, 
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    verbose=1, 
    restore_best_weights=True
)

# 모델 훈련
history = lstm_model.fit(
    X_train_lstm, y_train_lstm,
    epochs=100,
    batch_size=32,
    validation_data=(X_val_lstm, y_val_lstm),
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

## 5. 모델 평가

In [ ]:
# 학습 곡선 시각화
def plot_learning_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 손실 곡선
    ax1.plot(history.history['loss'], label='훈련 손실')
    ax1.plot(history.history['val_loss'], label='검증 손실')
    ax1.set_title('손실 곡선')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('손실')
    ax1.legend()
    
    # 정확도 곡선
    ax2.plot(history.history['accuracy'], label='훈련 정확도')
    ax2.plot(history.history['val_accuracy'], label='검증 정확도')
    ax2.set_title('정확도 곡선')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('정확도')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

plot_learning_curves(history)

In [ ]:
# 테스트 세트에서 모델 평가
test_loss, test_acc = lstm_model.evaluate(X_test_lstm, y_test_lstm, verbose=1)
print(f"테스트 손실: {test_loss:.4f}")
print(f"테스트 정확도: {test_acc:.4f}")

# 예측
y_pred = lstm_model.predict(X_test_lstm)
y_pred_classes = np.argmax(y_pred, axis=1)

# 혼동 행렬
conf_matrix = confusion_matrix(y_test_lstm, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=np.unique(y),
            yticklabels=np.unique(y))
plt.title('혼동 행렬')
plt.xlabel('예측 레이블')
plt.ylabel('실제 레이블')
plt.show()

# 분류 보고서
print("\n분류 보고서:")
print(classification_report(y_test_lstm, y_pred_classes))

## 6. 신경 상태 예측 및 시각화

In [ ]:
# 예측 확률 시각화
def plot_prediction_probabilities(model, X, y_true, class_names, n_samples=5):
    # 무작위 샘플 선택
    indices = np.random.choice(len(X), n_samples, replace=False)
    X_selected = X[indices]
    y_true_selected = y_true[indices]
    
    # 예측 확률 계산
    y_probs = model.predict(X_selected)
    
    # 플롯 생성
    fig, axes = plt.subplots(n_samples, 1, figsize=(12, 4*n_samples))
    
    for i, (probs, true_label, ax) in enumerate(zip(y_probs, y_true_selected, axes)):
        # 확률 막대 그래프
        bars = ax.bar(class_names, probs, color='skyblue')
        
        # 실제 레이블 강조
        bars[true_label].set_color('green')
        
        # 예측 레이블 표시
        pred_label = np.argmax(probs)
        if pred_label != true_label:
            bars[pred_label].set_color('red')
        
        ax.set_ylim([0, 1])
        ax.set_ylabel('확률')
        ax.set_title(f'샘플 {i+1}: 실제 레이블 = {class_names[true_label]}, '
                     f'예측 레이블 = {class_names[pred_label]}')
        
        # 확률값 표시
        for bar, prob in zip(bars, probs):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                   f'{prob:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# 클래스 이름 정의 (예시)
class_names = [f'상태 {i}' for i in range(num_classes)]

# 예측 확률 시각화
plot_prediction_probabilities(lstm_model, X_test_lstm, y_test_lstm, class_names, n_samples=5)

## 7. 적응형 전기자극 파라미터 추천 시스템

In [ ]:
# 상태별 최적 전기자극 파라미터 정의 (예시)
stimulation_parameters = {
    0: {
        'frequency': 20,  # Hz
        'amplitude': 2.5, # mA
        'pulse_width': 200, # µs
        'duration': 30,   # minutes
        'description': '낮은 강도 자극 - 초기 신경 손상 단계'
    },
    1: {
        'frequency': 50,
        'amplitude': 3.5,
        'pulse_width': 300,
        'duration': 45,
        'description': '중간 강도 자극 - 활성화 촉진 단계'
    },
    2: {
        'frequency': 100,
        'amplitude': 5.0,
        'pulse_width': 500,
        'duration': 60,
        'description': '높은 강도 자극 - 재생 가속화 단계'
    }
}

# 패턴 기반 추천 함수
def recommend_stimulation(model, signal_sequence, scaler, stimulation_parameters):
    # 신호 정규화
    signal_scaled = scaler.transform(signal_sequence)
    
    # LSTM 입력 형태로 변환
    samples, features = signal_scaled.shape
    signal_reshaped = signal_scaled.reshape(1, samples, features)
    
    # 상태 예측
    state_probs = model.predict(signal_reshaped)[0]
    predicted_state = np.argmax(state_probs)
    confidence = state_probs[predicted_state]
    
    # 추천 파라미터
    recommended_params = stimulation_parameters[predicted_state]
    
    return predicted_state, confidence, recommended_params

# 예시 시연 (무작위 샘플 사용)
sample_idx = np.random.randint(0, len(X_test_lstm))
sample_sequence = X_test_lstm[sample_idx]
true_state = y_test_lstm[sample_idx]

# 원래 형태로 변환 (2D)
sample_sequence_2d = sample_sequence.reshape(-1, sample_sequence.shape[-1])

# 추천 실행
predicted_state, confidence, recommended_params = recommend_stimulation(
    lstm_model, sample_sequence_2d, scaler, stimulation_parameters
)

# 결과 출력
print(f"샘플의 실제 상태: {true_state} ({class_names[true_state]})")
print(f"예측된 상태: {predicted_state} ({class_names[predicted_state]}) - 신뢰도: {confidence:.2f}")
print("\n추천 전기자극 파라미터:")
for key, value in recommended_params.items():
    print(f"{key}: {value}")

## 8. 모델 저장 및 내보내기

In [ ]:
# 모델 저장
model_save_path = "../models/lstm_neural_signal_classifier"
lstm_model.save(model_save_path)
print(f"모델이 {model_save_path}에 저장되었습니다.")

# 스케일러 저장
import joblib
scaler_save_path = "../models/lstm_signal_scaler.pkl"
joblib.dump(scaler, scaler_save_path)
print(f"스케일러가 {scaler_save_path}에 저장되었습니다.")

# 전기자극 파라미터 저장
import json
params_save_path = "../models/stimulation_parameters.json"
with open(params_save_path, 'w') as f:
    # 정수 키를 문자열로 변환
    str_params = {str(k): v for k, v in stimulation_parameters.items()}
    json.dump(str_params, f, indent=4)
print(f"전기자극 파라미터가 {params_save_path}에 저장되었습니다.")

## 9. 결론 및 다음 단계

이 노트북에서는 LSTM 기반 신경 신호 분류 모델을 구축하고 학습하였습니다. 다음 단계로 고려할 수 있는 사항들은 다음과 같습니다:

1. **하이퍼파라미터 최적화**: 그리드 서치 또는 베이지안 최적화를 통해 모델 하이퍼파라미터를 더 세밀하게 조정합니다.
2. **모델 앙상블링**: 여러 모델을 결합하여 예측 성능을 향상시킵니다.
3. **실시간 처리 구현**: 학습된 모델을 실시간 신호 처리 시스템에 통합합니다.
4. **전기자극 파라미터 최적화**: 피드백 루프를 통해 전기자극 파라미터를 자동으로 최적화합니다.
5. **임상 검증**: 실제 신경재생 환경에서 시스템의 효과를 검증합니다.

이 시스템은 신경 손상 후 재생 과정에서 맞춤형 전기자극을 제공함으로써 회복 속도와 효과를 향상시키는 데 기여할 수 있습니다.